In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, lit
import datetime
import pandas as pd
import random
import shutil
import os

In [2]:
# Configurar la sesión de Spark
spark = SparkSession.builder \
    .appName("Transformación datos parquet") \
    .getOrCreate()

    Leer documento SPARK

In [5]:
# Leer archivo Parquet
df = spark.read.parquet("./datos_generados/tabla_datos_mult.parquet")

# Muestra las primeras filas del DataFrame
df.show()

+-------+---------+--------+------+------------------+----------+--------------------+------+-------------+--------------------+------------+----------------------+---------------------+--------------------+---------------+--------------+--------------------+--------+----------------+----------------------+---------------------+-------+------+--------+----------------+---------+------+-----+----------------+------------------------+
|id_pais|id_ciudad|    Pais|Ciudad|       Coordenadas|      date|         dateChecked| death|deathIncrease|                hash|hospitalized|hospitalizedCumulative|hospitalizedCurrently|hospitalizedIncrease|inIcuCumulative|inIcuCurrently|        lastModified|negative|negativeIncrease|onVentilatorCumulative|onVentilatorCurrently|pending|posNeg|positive|positiveIncrease|recovered|states|total|totalTestResults|totalTestResultsIncrease|
+-------+---------+--------+------+------------------+----------+--------------------+------+-------------+-------------------

    Eliminar columnas

In [6]:
def eliminar_columnas(df):
    # Lista de columnas a eliminar
    columnas_a_eliminar = ['id_pais', 'id_ciudad', 'dateChecked', 'hash', 'lastModified', 'posNeg', 'recovered', 'total']
    # Filtrar solo las columnas que existen en el DataFrame para evitar errores
    columnas_a_eliminar = [col for col in columnas_a_eliminar if col in df.columns]
    # Eliminar las columnas
    df_resultante = df.drop(*columnas_a_eliminar)
    return df_resultante

In [7]:
df_nuevo = eliminar_columnas(df)
df_nuevo.show()

+--------+------+------------------+----------+------+-------------+------------+----------------------+---------------------+--------------------+---------------+--------------+--------+----------------+----------------------+---------------------+-------+--------+----------------+------+----------------+------------------------+
|    Pais|Ciudad|       Coordenadas|      date| death|deathIncrease|hospitalized|hospitalizedCumulative|hospitalizedCurrently|hospitalizedIncrease|inIcuCumulative|inIcuCurrently|negative|negativeIncrease|onVentilatorCumulative|onVentilatorCurrently|pending|positive|positiveIncrease|states|totalTestResults|totalTestResultsIncrease|
+--------+------+------------------+----------+------+-------------+------------+----------------------+---------------------+--------------------+---------------+--------------+--------+----------------+----------------------+---------------------+-------+--------+----------------+------+----------------+------------------------+
|

In [8]:
def transformar_datos(df):
    # Divide la columna 'date' en tres partes: año, mes y día
    df = df.withColumn("YYYY", split(col("date"), "-")[0]) \
           .withColumn("MM", split(col("date"), "-")[1]) \
           .withColumn("DD", split(col("date"), "-")[2])
    # Elimina la columna 'date' original
    df = df.drop("date")
    # Añade la columna 'fecha_procesado' con la fecha actual
    fecha_actual = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    df = df.withColumn("fecha_procesado", lit(fecha_actual))
    return df

In [9]:
df_n = transformar_datos(df_nuevo)
df_n.show()

+--------+------+------------------+------+-------------+------------+----------------------+---------------------+--------------------+---------------+--------------+--------+----------------+----------------------+---------------------+-------+--------+----------------+------+----------------+------------------------+----+---+---+----------------+
|    Pais|Ciudad|       Coordenadas| death|deathIncrease|hospitalized|hospitalizedCumulative|hospitalizedCurrently|hospitalizedIncrease|inIcuCumulative|inIcuCurrently|negative|negativeIncrease|onVentilatorCumulative|onVentilatorCurrently|pending|positive|positiveIncrease|states|totalTestResults|totalTestResultsIncrease|YYYY| MM| DD| fecha_procesado|
+--------+------+------------------+------+-------------+------------+----------------------+---------------------+--------------------+---------------+--------------+--------+----------------+----------------------+---------------------+-------+--------+----------------+------+----------------+

In [11]:
# --------------------------------------------------------------------------------#
# Transformar DataFrame de PySpark a DataFrame de Pandas
tabla_final_panda = df_n.toPandas()


# --------------------------------------------------------------------------------#
# Exportar archivo .parquet a la carpeta de 'datos_generados'
tabla_final_panda.to_parquet("./datos_generados/tabla_procesada.parquet")

In [12]:
# Lista de nombres de archivos para mover
nombres_archivos = ['ciudades.json', 'paises.json', 'tabla_datos_mult.parquet']

# Directorio original y destino
directorio_original = 'datos_generados'
directorio_destino = 'old'

# Asegurándonos de que la carpeta destino existe
os.makedirs(directorio_destino, exist_ok=True)

# Proceso de mover cada archivo
for nombre_archivo in nombres_archivos:
    ruta_original = os.path.join(directorio_original, nombre_archivo)
    ruta_destino = os.path.join(directorio_destino, nombre_archivo)

    try:
        shutil.move(ruta_original, ruta_destino)
        print(f"Archivo {nombre_archivo} movido exitosamente a {directorio_destino}!")
    except Exception as e:
        print(f"Ocurrió un error al mover el archivo {nombre_archivo}: {e}")

Archivo ciudades.json movido exitosamente a old!
Archivo paises.json movido exitosamente a old!
Archivo tabla_datos_mult.parquet movido exitosamente a old!
